### Describe the goals of the study.

The aim of this report is to study the current market situation for the Book publishing industry, in order to generate a value proposition for a new product.

We will study data from the following tables stored in a database:
- Books
- Authors
- Reviews
- Ratings
- Publishers

This report is laid out as follows:

**TABLE OF CONTENTS**
* 1. [Connect to Database](#chapter1)
* 2. [Study Data](#chapter2)
* 3. [Tasks](#chapter2)
    * 3.1 [Books after 2000](#chapter3.1)
    * 3.2 [User Reviews and Ratings](#chapter3.2)
    * 3.3 [Top Publisher](#chapter3.3)
    * 3.4 [Top Rated Authors](#chapter3.4)
    * 3.5 [Text Reviews](#chapter3.5)

### Connect to Database <a class="anchor" id="chapter1"></a>

In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student', # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # connection port
             'db': 'data-analyst-final-project-db'} # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Study Data <a class="anchor" id="chapter2"></a>

In [3]:
query = '''
            SELECT * FROM books LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Books table holds data about every book; it is connected to the rest of the tables with "author_id", "publisher_id" and "book_id" columns. Every entry is a book with its author id, title, number of pages, publication date and publisher id.

In [4]:
query = '''
            SELECT * FROM authors LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


The "authors" table contains the name and author_id of the authors.

In [5]:
query = '''
            SELECT * FROM publishers LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Publisher table contains publisher name and id.

In [6]:
query = '''
            SELECT * FROM ratings LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


The ratings table contains rating id, rating score, book id and the username who rated.

In [7]:
query = '''
            SELECT * FROM reviews LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


reviews table contains data about the text in the review, username who wrote it, the review_id and book_id.

## Tasks <a class="anchor" id="chapter3"></a>

### Find the number of books released after January 1, 2000. <a class="anchor" id="chapter3.1"></a>

In [8]:
query = '''
            SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01';
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


There are a total of 819 books released after January 1st, 2000.

### User Reviews and Ratings <a class="anchor" id="chapter3.2"></a>

Find the number of user reviews and the average rating for each book.

We join the tables books, reviews and ratings on the "book_id" column and at the same time we count the number of reviews and calculate the average rating per book (using book_id as identifier).
The resulting table is grouped and sorted by book_id in ascending order.

In [9]:
query = '''
            SELECT
            books.book_id,
            books.title,
            COUNT(reviews.review_id) AS "Number of User Reviews",
            ROUND(AVG(ratings.rating), 2) AS "Average Rating"
            FROM
            books
            INNER JOIN reviews ON reviews.book_id = books.book_id
            INNER JOIN ratings ON ratings.book_id = books.book_id
            GROUP BY books.book_id
            ORDER BY "Number of User Reviews" DESC
            LIMIT 10;
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,Number of User Reviews,Average Rating
0,948,Twilight (Twilight #1),1120,3.66
1,750,The Hobbit or There and Back Again,528,4.13
2,673,The Catcher in the Rye,516,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,480,4.29
5,75,Angels & Demons (Robert Langdon #1),420,3.68
6,301,Harry Potter and the Order of the Phoenix (Har...,375,4.19
7,779,The Lightning Thief (Percy Jackson and the Oly...,372,4.08
8,722,The Fellowship of the Ring (The Lord of the Ri...,370,4.39
9,79,Animal Farm,370,3.73


The resulting table displays a sample of 10 books (ordered by average rating in descending order) with their title, number of user reviews and their average rating score rounded to 2 decimals.
We can observe that the top average rating scores seem to have 4 user reviews.

### Top Publisher <a class="anchor" id="chapter3.3"></a>

Identify the publisher that has released the greatest number of books with more than 50 pages. 

In [10]:
query = '''
            SELECT
            publishers.publisher,
            COUNT(books.book_id) AS "Number of Books Published"
            FROM
            publishers
            INNER JOIN books ON books.publisher_id = publishers.publisher_id
            WHERE books.num_pages > 50
            GROUP BY publishers.publisher_id
            ORDER BY "Number of Books Published" DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,Number of Books Published
0,Penguin Books,42


The top publisher is Penguin Books, with 42 books released.

### Top Rated Authors <a class="anchor" id="chapter3.4"></a>

Identify the author with the highest average book rating (look only at books with at least 50 ratings)

In [11]:
query = '''
            SELECT
            authors.author,
            ROUND(AVG(ratings.rating), 2) AS "Average Book Rating"
            FROM
            authors
            INNER JOIN books ON books.author_id = authors.author_id
            INNER JOIN ratings ON ratings.book_id = books.book_id
            GROUP BY authors.author
            HAVING COUNT(ratings.rating_id) > 50
            ORDER BY "Average Book Rating" DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,author,Average Book Rating
0,J.K. Rowling/Mary GrandPré,4.29


The top authors are J. K. Rowling/Mary Grandpé amd Agatha Christie.

### Text Reviews <a class="anchor" id="chapter3.5"></a>

Find the average number of text reviews among users who rated more than 50 books

In [12]:
query = '''
            SELECT
            COUNT(reviews.review_id) AS "Number of Text Reviews"
            FROM reviews
            WHERE reviews.username in (
                SELECT ratings.username
                FROM ratings
                GROUP BY ratings.username
                HAVING COUNT(ratings.rating_id) > 50)
            GROUP BY reviews.username;
        '''
top_text_reviews = pd.io.sql.read_sql(query, con = engine)
top_text_reviews

,Number of Text Reviews
0,28
1,25
2,18
3,22
4,27
5,26


In [13]:
round(top_text_reviews['Number of Text Reviews'].mean(), 2)

24.33

The table above shows the only 6 users rated more than 50 books and their number of text reviews. We can say that the average number of text reviews among users that have rated more than 50 books is 24.33.